In [73]:
import pandas as pd
import numpy as np

In [74]:
leagues = ["pl", "ll", "bl", "sa", "l1"]
stats_types = ["standard", "shooting", "passing", "passing_types", "goal_shot_creation", "defense", "possession", "playing_time", "misc"]

In [75]:
def concatCommonCols(df1, df2):
    common_cols = list(set(df1.columns) & set(df2.columns))
    return pd.concat([df1[common_cols], df2[common_cols]])


In [76]:
old_stats = {}
new_stats = {}
all_stats = {}
for type in stats_types:
    old_stats[type] = pd.read_csv('data/old_fbref/'+type+'.csv')
    new_stats[type] = pd.read_csv('data/new_fbref/'+type+'.csv')
    all_stats[type] = concatCommonCols(old_stats[type], new_stats[type])

In [77]:
df = old_stats["passing_types"]
df

,league,season,team,player,nation,pos,age,born,90s,Att,...,Pass Types_Crs,Pass Types_TI,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Outcomes_Cmp,Outcomes_Off,Outcomes_Blocks,nextSeason
0,ENG-Premier League,2005,Arsenal,Abou Diaby,FRA,MF,19.0,1986.0,8.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006
1,ENG-Premier League,2005,Arsenal,Alex Song,CMR,"DF,MF",17.0,1987.0,2.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006
2,ENG-Premier League,2005,Arsenal,Alexander Hleb,BLR,MF,24.0,1981.0,16.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006
3,ENG-Premier League,2005,Arsenal,Arturo Lupoli,ITA,FW,18.0,1987.0,0.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006
4,ENG-Premier League,2005,Arsenal,Ashley Cole,ENG,DF,24.0,1980.0,9.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32413,ITA-Serie A,2016,Udinese,Silvan Widmer,SUI,DF,23.0,1993.0,27.7,NaN,...,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
32414,ITA-Serie A,2016,Udinese,Simone Scuffet,ITA,GK,20.0,1996.0,5.1,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
32415,ITA-Serie A,2016,Udinese,Stipe Perica,CRO,"FW,MF",21.0,1995.0,12.1,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
32416,ITA-Serie A,2016,Udinese,Sven Kums,BEL,MF,28.0,1988.0,20.3,NaN,...,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017


In [79]:
def getFbrefFromTransfers(stats, transfers_players, name):
    df = stats
    merged = pd.merge(transfers_players, df, left_on=['playerName', 'season'], right_on=['player', 'nextSeason'])
    del merged['season_y']
    merged = merged.rename(columns={'season_x': 'season'})
    merged['season'] = merged['season'].astype(int)
    merged.to_csv('data/transfers_stats/'+ name, index=False)

In [80]:
def getTransfersPlayers(league):
    players = pd.read_csv('data/players/'+league+'_players.csv')
    transfers = pd.read_csv('data/transfers/'+ league + '_transfers.csv')
    transfers_players = pd.merge(transfers, players[['playerLink', 'currAge', 'height', 'foot']], on="playerLink", how="left")
    return transfers_players

In [81]:
transfers_players = {}
for league in leagues:
    transfers_players[league] = getTransfersPlayers(league)

for league in leagues:
    for type in stats_types:
        getFbrefFromTransfers(new_stats[type], transfers_players[league], "new/" + league + '/' + type + '.csv')
